In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#install pygaggle before transformers (first create the index using pyserini then install pygaggle)
!pip install pygaggle==0.0.2
!pip install transformers==4.17.0

In [15]:
import transformers
import pygaggle
import pandas as pd
from tqdm import tqdm
print(transformers.__version__)

4.17.0


In [4]:
from pygaggle.rerank.base import Query, Text
from pygaggle.rerank.transformer import MonoT5
reranker =  MonoT5()

In [5]:
relevant_bm25=pd.read_csv("/content/drive/MyDrive/Touche/relevant_bm25.csv")


In [6]:
relevant_bm25.head()

,Unnamed: 0,title_id,title,doc_id,score,content
0,0,2,"Which is better, a laptop or a desktop?",clueweb12-0300tw-82-03945___9,11.1267,benefits offer laptop despite rush replace de...
1,1,2,"Which is better, a laptop or a desktop?",clueweb12-1002wb-75-11480___2,10.6442,patches upgrades simple remotely central cons...
2,2,2,"Which is better, a laptop or a desktop?",clueweb12-0608wb-66-03868___1,10.1275,desktop laptop tech homeforumscontact hijack...
3,3,2,"Which is better, a laptop or a desktop?",clueweb12-0006wb-17-30732___8,9.9483,gateways nv75s17u affordable offers memory ra...
4,4,2,"Which is better, a laptop or a desktop?",clueweb12-0001wb-72-30848___8,9.9420,desktop replacements usually larger heavier s...


In [17]:
new=[]
for title in tqdm(relevant_bm25["title"].unique()):
  passages= relevant_bm25.loc[relevant_bm25["title"]==title]
  texts = [ Text(p[1].content, {'docid': p[1].doc_id}, 0) for p in passages.iterrows()]
  query = Query(title)
  reranked = reranker.rerank(query, texts)
  for i in range(0, 1000):
    new.append({"doc_id":reranked[i].metadata["docid"], "Score":reranked[i].score, "title":title})

  0%|          | 0/50 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/transformers/models/t5/tokenization_t5.py:195: UserWarning: This sequence already has </s>. In future versions this behavior may lead to duplicated eos tokens being added.
  f"This sequence already has {self.eos_token}. In future versions this behavior may lead to duplicated eos tokens being added."
100%|██████████| 50/50 [37:18<00:00, 44.77s/it]


In [ ]:
df=pd.DataFrame(new)

In [19]:
df.to_csv("/content/drive/MyDrive/Touche/reranked.csv")

In [21]:
df.head()

,doc_id,Score,title
0,clueweb12-0300tw-82-03945___9,-4.033607,"Which is better, a laptop or a desktop?"
1,clueweb12-1002wb-75-11480___2,-11.896247,"Which is better, a laptop or a desktop?"
2,clueweb12-0608wb-66-03868___1,-0.312437,"Which is better, a laptop or a desktop?"
3,clueweb12-0006wb-17-30732___8,-6.825424,"Which is better, a laptop or a desktop?"
4,clueweb12-0001wb-72-30848___8,-1.026770,"Which is better, a laptop or a desktop?"
